# 天池大赛·印象盐城

## 特征工程

初赛提供2012年1月-2017年10月盐城分车型销量配置数据。
第一阶段需要参赛者预测2017年11月盐城分车型销量数据，第二阶段需要参赛者预测2017年12月盐城分车型销量数据。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('[new] yancheng_train_20171226.csv', low_memory=False)
test = pd.read_csv('yancheng_testA_20171225.csv')

In [3]:
test.head()

,predict_date,class_id,predict_quantity
0,201711,103507,NaN
1,201711,124140,NaN
2,201711,125403,NaN
3,201711,136916,NaN
4,201711,169673,NaN


### 一、异常值处理

1.1 发现数据里相同车型在同一月份里有两条记录，这里做简单加和

In [4]:
# labels = ['sale_date','class_id','sale_quantity']
labels = ['sale_date','class_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
          'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
          'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track']
# trainSale = train[labels]
train = train.groupby(labels).agg('sum').reset_index()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 31 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
compartment              20099 non-null int64
type_id                  20099 non-null int64
level_id                 20099 non-null object
department_id            20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null object
cylinder_number          20099 non-null int64
engine_t

1.2 三个功率为81/70的条目里共有两款车

    其中一款车有记载的功率为66和81，所以我们取平均值，令其为73
    
    另一款车有记载的功率为66，70,81和96，取平均值，令其为78

In [5]:
train[train['power']=='81/70']

,sale_date,class_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,...,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity,brand_id
12382,201601,175962,3,2,1,4,5,MT,1.6,L,...,1706,1469,1735,1275,5,2603,1460,1500,8,750
13450,201603,961962,3,2,1,4,5,MT,1.6,L,...,1706,1470,1740,1260,5,2603,1460,1500,5,750
13478,201604,175962,3,2,1,4,5,MT,1.6,L,...,1706,1469,1735,1275,5,2603,1460,1500,32,750


In [6]:
train.loc[12382,'power']=73
train.loc[13478,'power']=73
train.loc[13450,'power']=78
train[['power']] = train[['power']].astype('float32')

### 二、特征处理

2.1 品牌ID共有42个，进行One-hot编码

In [7]:
train['brand_id'].unique()

array([ 761,  106,   98,  836,   12,  814,  831,  750,  537,  450,  692,
        985,  841,  638,  872,  953,  304,  196,  783,  608,  637,   75,
        923, 1566,  497,  813,  290, 1846,  807,  864,  498,  236,  542,
        512,  294,   49, 1500,  126,  682,   68,   76, 1276], dtype=int64)

In [9]:
Brand_id_dummies = pd.get_dummies(train['brand_id'], prefix='brand_id')
train = train.drop(['brand_id'],axis=1)
train = pd.concat([train, Brand_id_dummies],axis=1)

,sale_date,class_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,...,brand_id_841,brand_id_864,brand_id_872,brand_id_923,brand_id_953,brand_id_985,brand_id_1276,brand_id_1500,brand_id_1566,brand_id_1846
0,201201,125403,2,3,2,2,6,AT,2.4,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,201201,125403,2,3,2,2,6,AT,2.4,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,201201,125403,2,3,2,2,6,AT,3.0,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,201201,136916,3,2,2,5,4,AT,2.0,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,201201,136916,3,2,2,5,4,AT,2.0,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,201201,136916,3,2,2,5,4,AT,2.0,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,201201,136916,3,2,2,5,5,AT,2.4,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,201201,136916,3,2,2,5,6,AT,2.5,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,201201,136916,3,2,2,5,6,AT,2.5,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,201201,136916,3,2,2,5,6,AT,2.5,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2.2 厢数有三种取值，进行One-hot编码

In [10]:
train['compartment'].unique()

array([2, 3, 1], dtype=int64)

In [11]:
compartment_dummies = pd.get_dummies(train['compartment'], prefix='compartment')
train = train.drop(['compartment'],axis=1)
train = pd.concat([train, compartment_dummies],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 74 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
type_id                  20099 non-null int64
level_id                 20099 non-null object
department_id            20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null float32
cylinder_number          20099 non-null int64
engine_torque            20099 non-null object
car_le

2.3 车型类别有四种

In [13]:
train['type_id'].unique()

array([3, 2, 4, 1], dtype=int64)

In [14]:
type_id_dummies = pd.get_dummies(train['type_id'], prefix='type_id')
train = train.drop(['type_id'],axis=1)
train = pd.concat([train, type_id_dummies],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 77 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
level_id                 20099 non-null object
department_id            20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null float32
cylinder_number          20099 non-null int64
engine_torque            20099 non-null object
car_length               20099 non-null int64
car_wi

2.4 车型级别有六种

In [16]:
train['level_id'].unique()

array(['2', '-', '1', '4', '3', '5'], dtype=object)

In [17]:
level_id_dummies = pd.get_dummies(train['level_id'], prefix='level_id')
train = train.drop(['level_id'],axis=1)
train = pd.concat([train, level_id_dummies],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 82 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
department_id            20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null float32
cylinder_number          20099 non-null int64
engine_torque            20099 non-null object
car_length               20099 non-null int64
car_width                20099 non-null int64
car_hei

2.5 车型系别有七种

In [19]:
train['department_id'].unique()

array([2, 5, 1, 3, 4, 6, 7], dtype=int64)

In [20]:
department_id_dummies = pd.get_dummies(train['department_id'], prefix='department_id')
train = train.drop(['department_id'],axis=1)
train = pd.concat([train, department_id_dummies],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 88 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null float32
cylinder_number          20099 non-null int64
engine_torque            20099 non-null object
car_length               20099 non-null int64
car_width                20099 non-null int64
car_height               20099 non-null int64
total_q

2.6 变速器档位有十种

In [22]:
train['TR'].unique()

array(['6', '4', '5', '8', '7', '0', '5;4', '8;7', '9', '1'], dtype=object)

In [23]:
TR_dummies = pd.get_dummies(train['TR'], prefix='TR')
train = train.drop(['TR'],axis=1)
train = pd.concat([train, TR_dummies],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 97 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null float32
cylinder_number          20099 non-null int64
engine_torque            20099 non-null object
car_length               20099 non-null int64
car_width                20099 non-null int64
car_height               20099 non-null int64
total_quality            20099 non-null int64
equipmen

2.7 变速器形式有七种

In [25]:
train['gearbox_type'].unique()

array(['AT', 'MT', 'DCT', 'CVT', 'MT;AT', 'AT;DCT', 'AMT'], dtype=object)

In [26]:
gearbox_type_dummies = pd.get_dummies(train['gearbox_type'], prefix='gearbox_type')
train = train.drop(['gearbox_type'],axis=1)
train = pd.concat([train, gearbox_type_dummies],axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 103 entries, sale_date to gearbox_type_MT;AT
dtypes: float32(1), float64(80), int64(17), object(5)
memory usage: 15.7+ MB


2.8 排量

In [27]:
train['displacement'].unique()

array([ 2.4,  3. ,  2. ,  2.5,  1. ,  1.1,  1.2,  1.5,  1.4,  1.6,  1.8,
        2.3,  1.9,  2.7,  2.8,  1.3,  3.6,  3.1,  0. ])